In [1]:
import numpy as np
import pandas as pd
import seaborn as sea
import matplotlib.pyplot as plt

from neuprint import Client
from neuprint import fetch_adjacencies

In [2]:
# Initialize client
#token = '' # From https://neuprint.janelia.org/account
c = Client('neuprint.janelia.org', dataset='manc:v1.0', token=token)
c.fetch_version()

'0.1.0'

In [3]:
# IDs of neurons of interest

# Web neuron
ids = [10107] # Left VNC

# Chief 9A neurons
chief_9A_ids = [100513, # L1
                13157, # L2
                14517, # L3
                165560, # R1
                12443, # R2
                12804] # R3

In [8]:
# Find postsynaptic neurons
neuron_df, conn_df = fetch_adjacencies(ids, None)

In [9]:
# Sum weights across neuropils and sort
conn_df = conn_df.groupby(['bodyId_pre','bodyId_post'])['weight'].sum().reset_index().sort_values('weight', ascending=False)

In [10]:
# Remove connections with weights < weight_thresh
weight_thresh = 5 
idx_drop = conn_df.index[conn_df.weight<weight_thresh]
conn_df = conn_df.drop(index=idx_drop) 

In [15]:
# Find type of partner neurons  
neuron_type = [] 
n_syn_9A = 0
for idx, row in conn_df.iterrows():
    neuron_type_temp = neuron_df.type[neuron_df.bodyId == conn_df.bodyId_post[idx]].item()
    if neuron_type_temp is None:
        neuron_type.append('Unknown')
    elif 'IN' in neuron_type_temp:
        neuron_type.append('IN')
        if conn_df.bodyId_post[idx] in chief_9A_ids:
            n_syn_9A += conn_df.weight[idx].item()
    elif 'MN' in neuron_type_temp:
        neuron_type.append('MN')
    elif 'AN' in neuron_type_temp:
        neuron_type.append('AN')
    elif 'DN' in neuron_type_temp:
        neuron_type.append('DN')
    elif 'SN' in neuron_type_temp or 'SA' in neuron_type_temp:
        neuron_type.append('SN')
    elif 'EN' in neuron_type_temp or 'EA' in neuron_type_temp:
        # The web neuron makes only few synapses with efferent neurons (ENs). They are counted as IN here.
        neuron_type.append('IN') 
        
conn_df['neuron_type'] = neuron_type

In [16]:
# Sum weights for each type 
conn_df_sum = conn_df.groupby(['neuron_type'])['weight'].sum().reset_index().sort_values('weight', ascending=False)
conn_df_sum

,neuron_type,weight
2,IN,10772
4,Unknown,958
3,MN,894
0,AN,702
1,DN,247


In [22]:
# Calculate relativee weight for specific type 
conn_df_sum.weight[conn_df_sum.neuron_type=='IN'].item() / conn_df_sum.weight.sum()

0.7936344212775363

In [14]:
n_syn_9A / conn_df_sum.weight.sum()

0.01930302807043395